In [3]:
pwd

'/Users/reem/Desktop/Research/DebiasMulticlassWordEmbedding/Debiasing'

## The cell below imports necessary code from Manzini et al. 

In [49]:
from scipy.stats import ttest_rel, spearmanr
import string
import numpy as np
from gensim.models.keyedvectors import Word2VecKeyedVectors
import json

def _unary_s(embeddings, target, attributes):
    return np.mean([ spatial.distance.cosine(embeddings[target], embeddings[ai]) for ai in attributes ])

def multiclass_evaluation(embeddings, targets, attributes):
    targets_eval = []
    for targetSet in targets:
        for target in targetSet:
            for attributeSet in attributes:
                targets_eval.append(_unary_s(embeddings, target, attributeSet))
    m_score = np.mean(targets_eval)
    return m_score, targets_eval

def load_analogy_templates(json_filepath, mode):
    with open(json_filepath, "r") as f:	
        loadedData = json.load(f)
        return loadedData["analogy_templates"][mode]
def load_test_terms(json_filepath):
    with open(json_filepath, "r") as f:	
        loadedData = json.load(f)
        return loadedData["testTerms"]
    
def load_eval_terms(json_filepath, mode):
    with open(json_filepath, "r") as f:	
        loadedData = json.load(f)
        return loadedData["eval_targets"], loadedData["analogy_templates"][mode].values()
    
def load_def_sets(json_filepath):
    with open(json_filepath, "r") as f: 
        loadedData = json.load(f)
        return {i: v for i, v in enumerate(loadedData["definite_sets"])}

def load_legacy_w2v(w2v_file, dim=50):
    vectors = {}
    with open(w2v_file, 'r') as f:
        for line in f:
            vect = line.strip().rsplit()
            word = vect[0]
            vect = np.array([float(x) for x in vect[1:]])
            if(dim == len(vect)):
                vectors[word] = vect
        
    return vectors, dim

def pruneWordVecs(wordVecs):
    newWordVecs = {}
    for word, vec in wordVecs.items():
        valid=True
        if(not isValidWord(word)):
            valid = False
        if(valid):
            newWordVecs[word] = vec
    return newWordVecs

def identify_bias_subspace(vocab, def_sets, subspace_dim, embedding_dim):
    """
    Similar to bolukbasi's implementation at
    https://github.com/tolga-b/debiaswe/blob/master/debiaswe/debias.py
    vocab - dictionary mapping words to embeddings
    def_sets - sets of words that represent extremes? of the subspace
            we're interested in (e.g. man-woman, boy-girl, etc. for binary gender)
    subspace_dim - number of vectors defining the subspace
    embedding_dim - dimensions of the word embeddings
    """
    # calculate means of defining sets
    means = {}
    for k, v in def_sets.items():
        wSet = []
        for w in v:
            try:
                wSet.append(vocab[w])
            except KeyError as e:
                pass
        set_vectors = np.array(wSet)
        means[k] = np.mean(set_vectors, axis=0)

    # calculate vectors to perform PCA
    matrix = []
    for k, v in def_sets.items():
        wSet = []
        for w in v:
            try:
                wSet.append(vocab[w])
            except KeyError as e:
                pass
        set_vectors = np.array(wSet)
        diffs = set_vectors - means[k]
        matrix.append(diffs)
    
    matrix = np.concatenate(matrix)

    pca = PCA(n_components=subspace_dim)
    pca.fit(matrix)

    return pca.components_

def project_onto_subspace(vector, subspace):
    v_b = np.zeros_like(vector)
    for component in subspace:
        v_b += np.dot(vector.transpose(), component) * component
    return v_b

def neutralize_and_equalize(vocab, words, eq_sets, bias_subspace, embedding_dim):
    """
    vocab - dictionary mapping words to embeddings
    words - words to neutralize
    eq_sets - set of equality sets
    bias_subspace - subspace of bias from identify_bias_subspace
    embedding_dim - dimensions of the word embeddings
    """

    if bias_subspace.ndim == 1:
        bias_subspace = np.expand_dims(bias_subspace, 0)
    elif bias_subspace.ndim != 2:
        raise ValueError("bias subspace should be either a matrix or vector")

    new_vocab = vocab.copy()
    for w in words:
        # get projection onto bias subspace
        if w in vocab:
            v = vocab[w]
            v_b = project_onto_subspace(v, bias_subspace)

            new_v = (v - v_b) / np.linalg.norm(v - v_b)
            #print np.linalg.norm(new_v)
            # update embedding
            new_vocab[w] = new_v

    normalize(new_vocab)

    for eq_set in eq_sets:
        mean = np.zeros((embedding_dim,))

        #Make sure the elements in the eq sets are valid
        cleanEqSet = []
        for w in eq_set:
            try:
                _ = new_vocab[w]
                cleanEqSet.append(w)
            except KeyError as e:
                pass

        for w in cleanEqSet:
            mean += new_vocab[w]
        mean /= float(len(cleanEqSet))

        mean_b = project_onto_subspace(mean, bias_subspace)
        upsilon = mean - mean_b

        for w in cleanEqSet:
            v = new_vocab[w]
            v_b = project_onto_subspace(v, bias_subspace)

            frac = (v_b - mean_b) / np.linalg.norm(v_b - mean_b)
            new_v = upsilon + np.sqrt(1 - np.sum(np.square(upsilon))) * frac

            new_vocab[w] = new_v

    return new_vocab

def isValidWord(word):
    return all([c.isalpha() for c in word])

def normalize(word_vectors):
    for k, v in word_vectors.items():
        word_vectors[k] = v / np.linalg.norm(v)

## Experiment starts here 

In [30]:
vocabPath = 'data/vocab/race_attributes_optm.json'
mode = 'role'
embeddingPath = 'data/w2vs/reddit.US.txt.tok.clean.cleanedforw2v.w2v'
k = 1

#### Code below from debias.py uses all functions listed above

In [6]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
outprefix = vocabPath.replace("/", "_").replace("\\", "_").replace(".", "_")


print("Loading vocabulary from {}".format(vocabPath))

analogyTemplates = load_analogy_templates(vocabPath, mode)
defSets = load_def_sets(vocabPath)
testTerms = load_test_terms(vocabPath)

neutral_words = []
for value in analogyTemplates.values():
    neutral_words.extend(value)

print("Loading embeddings from {}".format(embeddingPath))
word_vectors, embedding_dim = load_legacy_w2v(embeddingPath)

print("Pruning Word Vectors... Starting with", len(word_vectors))
word_vectors = pruneWordVecs(word_vectors)
print("\tEnded with", len(word_vectors))


Loading vocabulary from data/vocab/race_attributes_optm.json
Loading embeddings from data/w2vs/reddit.US.txt.tok.clean.cleanedforw2v.w2v
Pruning Word Vectors... Starting with 44895
	Ended with 44895


In [41]:
testTerms

['chair', 'house', 'smart', 'criminal', 'executive', 'farmer']

In [131]:
analogyTemplates 

{'caucasian': ['manager',
  'executive',
  'redneck',
  'hillbilly',
  'leader',
  'farmer'],
 'asian': ['doctor', 'engineer', 'laborer', 'teacher'],
 'black': ['slave', 'musician', 'runner', 'criminal', 'homeless']}

In [42]:
defSets

{0: ['black', 'caucasian', 'asian'],
 1: ['african', 'caucasian', 'asian'],
 2: ['black', 'white', 'asian'],
 3: ['africa', 'america', 'asia'],
 4: ['africa', 'america', 'china'],
 5: ['africa', 'europe', 'asia']}

In [56]:
neutral_words

['manager',
 'executive',
 'redneck',
 'hillbilly',
 'leader',
 'farmer',
 'doctor',
 'engineer',
 'laborer',
 'teacher',
 'slave',
 'musician',
 'runner',
 'criminal',
 'homeless']

##### Purpose of code below: Visualize the words distance in a 3D plot using TSNE because it is a non-linear metric on the glove twitter data set

Code from https://towardsdatascience.com/visualizing-word-embedding-with-pca-and-t-sne-961a692509f5

In [71]:
def display_tsne_scatterplot(word_vectors=None, input_words=None, words=None, label=None, color_map=None, perplexity = 0, learning_rate = 0, iteration = 0, topn=4, sample=18):
    
    # Get word vectors for words
    word_vectors = np.array([word_vectors[w] for w in words])
    analogyTerms = [word for word in words if word not in input_words ]
#     print(words)
    
    # Initalzie TSNE and fit word_embeddings
    #2D
    two_dim = TSNE(n_components = 2, random_state=0, perplexity = perplexity, learning_rate = learning_rate, n_iter = iteration).fit_transform(word_vectors)[:,:2]
    # 3D
#     three_dim = TSNE(n_components = 3, random_state=0, perplexity = perplexity, learning_rate = learning_rate, n_iter = iteration).fit_transform(word_vectors)[:,:3]

    
    data = []
    count = 0
        
    for i in range (len(input_words)):
#         print(input_words[i])
        # 2D
        trace = go.Scatter(
            x = two_dim[count:count+topn,0], 
            y = two_dim[count:count+topn,1],  
            text = words[count:count+topn],
            name = input_words[i],
            textposition = "top center",
            textfont_size = 20,
            mode = 'markers+text',
            marker = {
                'size': 10,
                'opacity': 0.8,
                'color': 2
            }

        )
        # 3D
#         trace = go.Scatter3d(
#             x = three_dim[count:count+topn,0], 
#             y = three_dim[count:count+topn,1],  
#             z = three_dim[count:count+topn,2], 
#             text = words[count:count+topn],
#             name = input_words[i],
#             textposition = "top center",
#             textfont_size = 20,
#             mode = 'markers+text',
#             marker = {
#                 'size': 10,
#                 'opacity': 0.8,
#                 'color': 3
#             }

#         )
#         print(trace)

        data.append(trace)
        count = count+topn
    
    trace_input = go.Scatter(
                    x = two_dim[count:,0], 
                    y = two_dim[count:,1],  
                    text = input_words,
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 1,
                        'color': 'black'
                    })
    # 3D
#     trace_input = go.Scatter3d(
#                     x = three_dim[count:,0], 
#                     y = three_dim[count:,1],  
#                     z = three_dim[count:,2],
#                     text = input_words,
#                     name = 'input words',
#                     textposition = "top center",
#                     textfont_size = 20,
#                     mode = 'markers+text',
#                     marker = {
#                         'size': 10,
#                         'opacity': 1,
#                         'color': 'black'
#                     }
#                     )
#     print('trace_input')
#     print(trace_input)
    data.append(trace_input)

# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5, 
        font=dict(
            family="Courier New",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Courier New ",
            size = 15),
        autosize = False,
        width = 1000,
        height = 1000
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

In [72]:
import gensim.downloader as api
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from sklearn.manifold import TSNE
import numpy as np


# Try with smaller number of word vectors first 
keys = np.random.choice(list(word_vectors.keys()), 100)
word_vectors_sample = {word:word_vectors[word] for word in keys}

# Define classes
classes = list(analogyTemplates.keys())

# Find classes and analogyTerms vectors 
word_vectors_sample = {word:word_vectors[word] for word in sum([classes, neutral_words], [])}


### Visualize biased word embeddings

In [73]:
# Plot using word_vectors 
display_tsne_scatterplot(word_vectors=word_vectors_sample, input_words=classes, words=neutral_words, perplexity= 50,  learning_rate = 500, iteration =10000)


### Visualize Hard Debiased word embeddings

In [65]:
# Redefine defSet to the only one we are using 
defSets_sample = {0: ['black', 'caucasian', 'asian']}

In [66]:
print("Identifying bias subspace")
subspace = identify_bias_subspace(word_vectors_sample, defSets_sample, k, embedding_dim)[:k]
# subspace_sample = identify_bias_subspace(word_vectors_sample, defSets, k, embedding_dim)[:k]

print("Neutralizing and Equalizing")
new_hard_word_vectors = neutralize_and_equalize(word_vectors_sample, neutral_words,
                    defSets_sample.values(), subspace, embedding_dim)


Identifying bias subspace
Neutralizing and Equalizing


In [70]:
print("Visualize Debiased word embeddings")
display_tsne_scatterplot(word_vectors=new_hard_word_vectors, input_words=classes, words=neutral_words, perplexity= 2,  learning_rate = 500, iteration =10000)


Visualize Debiased word embeddings


In [68]:
print("Identifying bias subspace")
subspace = identify_bias_subspace(word_vectors, defSets, k, embedding_dim)[:k]
# subspace_sample = identify_bias_subspace(word_vectors_sample, defSets, k, embedding_dim)[:k]

print("Neutralizing and Equalizing")
new_hard_word_vectors = neutralize_and_equalize(word_vectors, neutral_words,
                    defSets.values(), subspace, embedding_dim)
display_tsne_scatterplot(word_vectors=new_hard_word_vectors, input_words=classes, words=neutral_words, perplexity= 5,  learning_rate = 500, iteration =10000)


Identifying bias subspace
Neutralizing and Equalizing


### Visualize Double Hard Debiased word embeddings

In [ ]:
print("Visualize Debiased word embeddings")
display_tsne_scatterplot(word_vectors=double_hard_word_vectors, input_words=classes, words=neutral_words, perplexity= 5,  learning_rate = 500, iteration =10000)

In [ ]:

print("Performing Evaluation")
evalTargets, evalAttrs = load_eval_terms(vocabPath, mode)

print("Biased Evaluation Results")
biasedMAC, biasedDistribution = multiclass_evaluation(word_vectors_sample, evalTargets, evalAttrs)
print("Biased MAC:", biasedMAC)


print("HARD Debiased Evaluation Results")
debiasedMAC, debiasedDistribution = multiclass_evaluation(new_hard_word_vectors, evalTargets, evalAttrs)
print("HARD MAC:", debiasedMAC)

statistics, pvalue = ttest_rel(biasedDistribution, debiasedDistribution)
print("HARD Debiased Cosine difference t-test", pvalue)

In [ ]:

# # Plot using glove 
# model = api.load("glove-twitter-25")
# print("Visualize biased word embeddings")
# display_tsne_scatterplot_3D(model=model, perplexity= 5,  learning_rate = 500, iteration =10000)
# print("Visualize Debiased word_embeddings")
# # NEED TO GET WORD_VECTORS FROM GLVOE
# # word_vectors, embedding_dim = load_legacy_w2v(model)
# word_vectors = pruneWordVecs(word_vectors)
# subspace = identify_bias_subspace(word_vectors, defSets, args.k, embedding_dim)[:args.k]
# new_hard_word_vectors = neutralize_and_equalize(word_vectors, neutral_words,
#                     defSets.values(), subspace, embedding_dim)
# display_tsne_scatterplot_3D(word_vectors=new_hard_word_vectors, perplexity= 5,  learning_rate = 500, iteration =10000)

# # Plot works with glove but needs filtering 
# model = api.load("glove-twitter-25")
# # model = '/Users/reem/gensim-data/glove-twitter-25/glove-twitter-25.gz'
# print("Visualize biased word embeddings")
# display_tsne_scatterplot_3D(model=model, perplexity= 5,  learning_rate = 500, iteration =10000)